# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [25]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [26]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [27]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

scroll_down(browser,SCROLL_PAUSE_TIME)

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


In [28]:
def scroll_down(self,pause_time):
    last_height = self.execute_script('return document.body.scrollHeight')
    while True:
        self.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(pause_time)
        new_height = self.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            print('到達頁面底端')
            break
        else:
            print('網頁更新中...')
        last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [29]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [30]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] ebay擬售分類廣告業務　估值約3000億元

▲eBay打算出售分類廣告業務。（圖／達志影像／美聯社）
記者陳心怡／外電報導
根據《華爾街日報》報導，美國電商網站eBay表示，為了應對投資者的壓力，公司正在與多方買家接觸，打算出售分類廣告業務，該業務的價值可能約為100億美元（約新台幣3,073億元），而公司未來將透過專注於線上市場業務來改善業績。
eBay臨時首席執行長斯申克爾（Scott Schenkel）在一份聲明中表示，eBay董事會和管理階層正致力於最大化公司的分類廣告業務價值，並重新定位公司的市場業務。
去年11月，該公司宣布以40.5億美元價格向全球知名票券平台Viagogo出售了在門票交易公司StubHub。如果這次eBay成功出售分類廣告業務，集團核心資產只剩下電商平台而已。
----------------------------------------------------------------------

[1] FinTech席捲全台金融業　中壽出招搶數位金融人才

▲中壽不斷精進數位實力，總經理黃淑芬（左五）更與AI新銳對談，讓青年勇於發聲。（圖／中壽提供）
記者紀佳妘／台北報導
金融科技浪潮席捲金融業，中國人壽今（24）日表示，邀請「AI新銳領航者競賽」前3名學生團體與中壽總經理面對面座談，跨界分享彼此心得，從公司內部科技創新團隊顧問文化，到參與外部AI新銳校園人才拔擢活動，多管齊下開創金融科技服務新藍海，搶更多的數位金融人才。
中國人壽總經理黃淑芬表示，科技為金融業帶來創新思維，民眾更重視科技帶來嶄新的保險體驗與服務，公司一直以愛與關懷的核心精神，透過數位力全面提升業務動能，並以實際行動支持政府數位化政策，並由內而外推動數位創新，集所有力量讓同仁化身為壽險鋼鐵人，其中外在部分，透過科技賦能，提升服務效率，內在則始終不變的是溫度，打造出的服務價值。
黃淑芬在座談會上鼓勵學子們善用所學接軌知識與實務，面對未來趨勢更要勇於驅動產業創新，運用AI找出的關鍵痛點，邁向頂尖數據人才。一名得獎同學指出，以前都是從課本練習，這是第一次接觸到真實的海量數據，發現除了跑數據，更重要的是這些數字背後